In [3]:
import seaborn as sns
from sklearn.metrics import pairwise

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 

/Users/dubeya/git/multi-eurlex/bert-venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
BERT_MODEL = "https://tfhub.dev/google/experts/bert/wiki_books/2" # @param {type: "string"} ["https://tfhub.dev/google/experts/bert/wiki_books/2", "https://tfhub.dev/google/experts/bert/wiki_books/mnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qqp/2", "https://tfhub.dev/google/experts/bert/wiki_books/squad2/2", "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2",  "https://tfhub.dev/google/experts/bert/pubmed/2", "https://tfhub.dev/google/experts/bert/pubmed/squad2/2"]


In [5]:
PREPROCESS_MODEL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"


In [6]:
sentences = [
  "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.",
  "The album went straight to number one on the Norwegian album chart, and sold to double platinum.",
  "Among the singles released from the album were the songs \"Be My Lover\" and \"Hard To Stay Awake\".",
  "Riccardo Zegna is an Italian jazz musician.",
  "Rajko Maksimović is a composer, writer, and music pedagogue.",
  "One of the most significant Serbian composers of our time, Maksimović has been and remains active in creating works for different ensembles.",
  "Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum",
  "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.",
  "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth.",
]


In [7]:
preprocess=hub.load(PREPROCESS_MODEL)

2024-08-14 14:05:49.815819: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-08-14 14:05:49.815843: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-08-14 14:05:49.815848: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-08-14 14:05:49.815862: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-14 14:05:49.815875: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-08-14 14:05:50.383852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [9]:
inputs=preprocess(sentences)

In [21]:
bert = hub.load(BERT_MODEL)


In [32]:
sentences=["potato", "patato"]
inputs=preprocess(sentences)
outputs = bert(inputs)
outputs

{'pooled_output': <tf.Tensor: shape=(2, 768), dtype=float32, numpy=
 array([[-0.25502983, -0.03921434,  0.2813091 , ..., -0.5237494 ,
         -0.95304567,  0.23198834],
        [-0.26120892,  0.8775287 ,  0.22002217, ...,  0.25623247,
         -0.7251264 , -0.18863827]], dtype=float32)>,
 'sequence_output': <tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
 array([[[-0.26078525, -0.03923446,  0.28910312, ..., -0.58149254,
          -1.8639846 ,  0.23628983],
         [-0.28190726, -0.2943976 ,  0.0175595 , ..., -0.43296713,
          -1.9986787 ,  0.55163753],
         [-0.17459384, -0.23282579, -0.22371535, ..., -0.5484337 ,
          -2.0260305 ,  0.5399683 ],
         ...,
         [-0.19457641, -0.25967982, -0.21225777, ..., -0.52961165,
          -2.0298033 ,  0.5441205 ],
         [-0.17740722, -0.25272688, -0.22287118, ..., -0.5387356 ,
          -2.027965  ,  0.5514798 ],
         [-0.17408606, -0.25458118, -0.22545046, ..., -0.5378515 ,
          -2.0283687 ,  0.55145043

In [71]:
from datasets import Dataset, DatasetDict


In [118]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [119]:
def load_custom_dataset(
    json_path="/Users/dubeya/git/data/carboncloud_raw_customer_db.product_db.json",
):
    data_json = json.load(open(json_path))
    converted_json = []
    for dict_file in data_json:
        dict_file["ingredient_name"].reverse()
        converted_json.append(
            {
                "text": dict_file["product_name"]
                + "\n"
                + dict_file["product_description"]
                + "\n"
                + " ".join(dict_file["ingredient_name"]),
                "label": dict_file["category"]["category_name"],
            }
        )
    dfItem = pd.DataFrame.from_records(converted_json)
    category_int_id = pd.DataFrame(dfItem.label.unique())
    category_int_dict = {}
    for index, row in category_int_id.iterrows():
        category_int_dict.update({row[0]: index})
    dfItem.label = dfItem.label.replace(category_int_dict)

    with open("category_int_mapping.txt", "w") as convert_file:
        convert_file.write(json.dumps(category_int_dict))

    mask = np.random.rand(dfItem.__len__()) < 0.8
    df_train_val = dfItem.iloc[mask, :]
    df_test = dfItem.iloc[~mask, :]
    mask = np.random.rand(df_train_val.__len__()) < 0.8
    df_train = df_train_val.iloc[mask, :]
    df_val = df_train_val.iloc[~mask, :]
    return (
        Dataset.from_pandas(df_train),
        Dataset.from_pandas(df_val),
        Dataset.from_pandas(df_test),
    )


In [120]:
td_t,_ ,_ = load_custom_dataset()

/var/folders/qm/lt6fypzx7qsg_gpwq9m1rv_w0000gn/T/ipykernel_2509/2783644624.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfItem.label = dfItem.label.replace(category_int_dict)


In [122]:
td_t['__index_level_0__']

[505,
 609,
 105,
 485,
 201,
 450,
 194,
 736,
 536,
 663,
 20,
 570,
 659,
 724,
 651,
 429,
 657,
 361,
 25,
 722,
 154,
 502,
 141,
 539,
 529,
 115,
 29,
 488,
 22,
 160,
 482,
 573,
 665,
 111,
 173,
 55,
 13,
 584,
 306,
 213,
 44,
 208,
 364,
 575,
 630,
 259,
 134,
 680,
 518,
 245,
 229,
 687,
 311,
 212,
 548,
 479,
 165,
 97,
 455,
 469,
 389,
 640,
 648,
 577,
 270,
 464,
 47,
 18,
 504,
 424,
 540,
 89,
 250,
 375,
 465,
 155,
 66,
 635,
 150,
 49,
 446,
 719,
 164,
 597,
 137,
 211,
 112,
 598,
 608,
 725,
 522,
 692,
 672,
 589,
 303,
 453,
 285,
 80,
 694,
 650,
 262,
 272,
 703,
 384,
 179,
 394,
 236,
 219,
 204,
 335,
 496,
 146,
 580,
 2,
 129,
 447,
 330,
 681,
 172,
 525,
 233,
 378,
 393,
 205,
 267,
 622,
 310,
 381,
 409,
 454,
 395,
 507,
 611,
 273,
 481,
 118,
 625,
 574,
 203,
 594,
 735,
 600,
 555,
 183,
 23,
 99,
 53,
 643,
 251,
 206,
 178,
 346,
 641,
 596,
 592,
 59,
 339,
 316,
 317,
 1,
 301,
 166,
 349,
 207,
 277,
 682,
 65,
 664,
 669,
 730,
 46

array([ True,  True,  True,  True, False,  True, False,  True,  True,
        True])

,text,label
0,Everyday Favourites Premium Sponge Mix\nCake/S...,0
1,Everyday Favourites Kids Spaghetti Bolognaise\...,1
2,Coronet Assorted Luxury Mini Pack Biscuits\nHo...,2
3,Everyday Favourites Seasoned Wedges\nWedges - ...,3
4,Everyday Favourites Carrot Cake (Pre-cut)\nCak...,4
...,...,...
731,Everyday Favourites Diced Mixed Peppers\nPeppe...,210
732,Individually Quick Frozen Sweetcorn\nCorn - Su...,211
733,Everyday Favourites Chopped Leaf Spinach\nLeaf...,185
735,Everyday Favourites Cut Leeks\nRoot vegetables...,213


In [90]:
dfItem

,text,label
0,Everyday Favourites Premium Sponge Mix\nCake/S...,0
1,Everyday Favourites Kids Spaghetti Bolognaise\...,1
2,Coronet Assorted Luxury Mini Pack Biscuits\nHo...,2
3,Everyday Favourites Seasoned Wedges\nWedges - ...,3
4,Everyday Favourites Carrot Cake (Pre-cut)\nCak...,4
...,...,...
733,Everyday Favourites Chopped Leaf Spinach\nLeaf...,185
734,Everyday Favourites Shredded Green Cabbage\nBr...,212
735,Everyday Favourites Cut Leeks\nRoot vegetables...,213
736,Farmstead Whole Gammon\nGammon Joints\nPork Wa...,170


In [87]:
category_int_dict

{'Baking mix': 0,
 'Pasta, cooked or otherwise prepared': 1,
 'Biscuits and cookies': 2,
 'Fried potatoes': 3,
 'Sweet pies and pastries': 4,
 'Cakes': 5,
 'Sweet bakery products': 6,
 'Boiled potatoes': 7,
 'Egg and oil based sauces and dressings': 8,
 'Plant based protein sources, prepared or preserved': 9,
 'Potato gratins, mashes, and similar': 10,
 'Butter': 11,
 'Egg based ready meals': 12,
 'E950-E969 - Sweeteners': 13,
 'Bread and other bakery products': 14,
 'Beef, minced or stewing, frozen': 15,
 'Lamb cuts boneless': 16,
 'Chips': 17,
 'Jams, marmalades, fruit or nut purées': 18,
 'Pineapples, prepared or preserved': 19,
 'Cheddar cheese': 20,
 'Sausages': 21,
 'Sweet buns': 22,
 'Oil based sauces and dressings': 23,
 'Sauces without animal products': 24,
 'Sauces with with animal products': 25,
 'Mustard': 26,
 'Apples, prepared or preserved': 27,
 'Soup and broth powder': 28,
 'Horseradish, fresh': 29,
 'Grana Padano': 30,
 'Other hard bread': 31,
 'Spice mixes': 32,
 'Ora

In [35]:
train_dataset = load_dataset(
        "multi_eurlex",
        language="en",
        label_level='level_1',
        split="train",
    )

In [72]:
td=

In [77]:
preprocess([td[0]['text']])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [54]:
lis=['hello', 'you']


In [61]:
lis.reverse()
' '.join(lis)

'you hello'

In [56]:
lis

['you', 'hello']